<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective" data-toc-modified-id="Objective-1">Objective</a></span></li><li><span><a href="#Back-propagation-(a.k.a-The-Chain-Rule)" data-toc-modified-id="Back-propagation-(a.k.a-The-Chain-Rule)-2">Back-propagation (a.k.a The Chain Rule)</a></span></li><li><span><a href="#(Binary)-Cross-Entropy-Loss" data-toc-modified-id="(Binary)-Cross-Entropy-Loss-3">(Binary) Cross Entropy Loss</a></span></li><li><span><a href="#Activations" data-toc-modified-id="Activations-4">Activations</a></span></li><li><span><a href="#Linear-Layer" data-toc-modified-id="Linear-Layer-5">Linear Layer</a></span></li><li><span><a href="#Putting-It-All-Together" data-toc-modified-id="Putting-It-All-Together-6">Putting It All Together</a></span></li><li><span><a href="#Our-Evaluation-Metric" data-toc-modified-id="Our-Evaluation-Metric-7">Our Evaluation Metric</a></span></li><li><span><a href="#Trainer" data-toc-modified-id="Trainer-8">Trainer</a></span></li><li><span><a href="#Pre-process-Data" data-toc-modified-id="Pre-process-Data-9">Pre-process Data</a></span></li><li><span><a href="#Datasets-&amp;-DataLoaders" data-toc-modified-id="Datasets-&amp;-DataLoaders-10">Datasets &amp; DataLoaders</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-11">Train</a></span></li><li><span><a href="#Comparison-with-sklearn" data-toc-modified-id="Comparison-with-sklearn-12">Comparison with sklearn</a></span></li><li><span><a href="#Adding-a-Second-Layer" data-toc-modified-id="Adding-a-Second-Layer-13">Adding a Second Layer</a></span></li><li><span><a href="#PyTorch-Comparison" data-toc-modified-id="PyTorch-Comparison-14">PyTorch Comparison</a></span></li></ul></div>

In [1]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer, load_digits
from sklearn.model_selection import train_test_split

In [2]:
# Set seed for reproducibility
seed = 9
np.random.seed(seed)

# Objective 

The goal of this notebook is to build a simple binary logistic classifier as a neural network. The network will consist of a single linear layer followed be a sigmoid activation and binary cross entropy as the loss function. When we're done we'll see that we're able to identify malignant tumors on sklearn's breast cancer dataset with 90% accuracy in just 10 epochs. 

We'll begin by deriving the back-propagation equations by hand for our particular scenario. In the process we'll realize that what we've done will generalize immediately to networks with an arbitrary number of layers, meaning we'll have built a framework that can model __any__ feedforward neural network. All by starting with binary logistic classifier.

# Back-propagation (a.k.a The Chain Rule)

We can view our logistic regression network as the composition of three functions

$$x \to \text{BCE} \circ \text{Sigmoid} \circ \text{Linear}(x)$$

While the loss function is not usually viewed as a layer of the network, treating it as the final layer makes computing the gradients easier. Let's denote the output of the $i$-th layer by $x_i$ so that

\begin{align}
    x_1 &= \text{Linear}(x)     \\
    x_2 &= \text{Sigmoid}(x_1)  \\
    x_3 &= \text{BCE}(x_2)
\end{align}

Since the last output we have is $x_3 = \text{BCE}(x_2)$, the first gradient we have to compute is the gradient of $\text{BCE}$ with respect to $x_2$

$$\frac{\partial \text{BCE}}{\partial x_2} = \frac{\partial \text{BCE}}{\partial x_2}(x_2) $$

Next, we have $x_2 = \text{Sigmoid}(x_1)$. The chain rule tells us

$$\frac{\partial \text{BCE}}{\partial x_1} = \frac{\partial \text{BCE}}{\partial x_2}(x_2) \times \frac{\partial \text{Sigmoid}}{\partial x_1}(x_1)$$

Last we have $x_1 = \text{Linear}(x)$, so the final gradient is

$$\frac{\partial \text{BCE}}{\partial x} = \frac{\partial \text{BCE}}{\partial x_1}(x_1) \times \frac{\partial \text{Linear}}{\partial x_1}(x)$$

Notice something peculiar? The first gradient we computed--the gradient with respect to the Sigmoid layer--is used to compute the next gradient--the gradient with respect to the preceding Linear layer. To compute the gradients of any network, we simply start at the final layer and successively pass the gradient back to the previous layer. That's why it's called backpropagation and not "just the chain rule". It really is helpful to envision passing the gradients backwards through the network like a baton.

__Note:__ So far we've been treating the input $x$ as a single variable, but most of the time $x$ will have more than one dimension. Don't worry, computing the gradients in the multi-variate case is more or less the same (though it involves something called the Jacobian--but we'll pretend we didn't hear that).

--------

# (Binary) Cross Entropy Loss

The only difference between binary cross entropy and cross entropy is that cross entropy requires an output probability for every single class, whereas binary cross entropy requires just a single output probability--the probability of the positive class. This is slightly annoying, because it means their derivatives have different functional forms. We'll stick with binary cross entropy.

Binary cross entropy penalizes predictions by the logarithm of their probabilities $\hat{y}$ (which we called $x_2$ above):

$$\text{BCE}(y, \hat{y}) = -y \ln(\hat{y}) - (1 - y)\ln(1 - \hat{y})$$

After simplifying, you'll find its derivative is

$$\frac{\partial \text{BCE}}{\partial \hat{y}} = \frac{\hat{y} - y}{\hat{y}(1 - \hat{y})}$$

Notice that we'll need to cache the labels $y$ and probabilities $\hat{y}$ to compute the backward pass. Since this is the first gradient we compute, there's no previous gradient to take in. However, the next gradient (belonging to the Sigmoid layer) will receive the above BCE gradient.  

In [3]:
class BinaryCrossEntropy:
    """Container for the forward and backward pass of BCE."""
    
    def forward(self, y, y_hat):
        """Return binary cross entropy given targets and predictions."""
        self.y, self.y_hat, = y, y_hat.clip(min=1e-8, max=1-1e-8)
        return np.where(y==1, -np.log(self.y_hat), -np.log(1 - self.y_hat))
    
    def backward(self):
        """Backpropagate the gradient with respect to soft predictions."""
        print(f"{self.y[:5]= }\n{self.y_hat[:5]= }")
        return (self.y_hat - self.y) / (self.y_hat * (1 - self.y_hat))

-----

# Activations

The easiest components to handle are the activation functions. Our activation is Sigmoid, which you'll often see defined as one of 

$$\text{Sigmoid}(x) = \frac{1}{1 + \text{exp}(-x)} \quad \text{or} \quad \frac{\text{exp}(x)}{1 + \text{exp}(x)}$$

It turns out that we need both versions to implement a numerically stable version of Sigmoid. Notice how when $x$ is very negative, $\text{exp}(-x)$ is incredibly large, and when $x$ is very positive $\text{exp}(x)$ is incredibly large--in both cases, too large to store in memory. The easy fix is to use the first version when $x > 0$ and the second when $x < 0$.

After simplifying, you'll find the derivative of $\text{Sigmoid}$ is

$$\frac{\partial \text{Sigmoid}}{\partial x} = \text{Sigmoid}(x)(1 - \text{Sigmoid}(x))$$

You may have noticed something peculiar when comparing this derivative to that of BCE. Since we denoted $\hat{y}$ to be the output of the Sigmoid layer, the derivative of sigmoid is exactly the same as the denominator in the derivative of BCE, so the two terms will cancel when multiplied (which is exactly what the chain rule tells us will happen). This is the reason you see functions like PyTorch's `binary_cross_entropy_with_logits`, which skips the Sigmoid activation and computes BCE directly from the inputs to the Sigmoid layer (which people call logits). It's more efficient computationally to do both in one go. However, since we're just trying to get our hands dirty to understand how networks work, we won't worry about optimizing things.

In [4]:
class Sigmoid:
    """Container for the forward and backward pass of sigmoid."""
    
    def forward(self, x):
        """Pass a mini-batch through a sigmoid layer."""
        self.y_hat = np.where(x > 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
        return self.y_hat
        
    def backward(self, grad):
        """Backpropagate the gradient given the previous gradient."""
        return self.y_hat * (1 - self.y_hat) * grad

----------

# Linear Layer

The last component we need to implement is the linear layer, which contains weights and biases. Denoting the output of this layer by $z$, we have

$$z = \text{Linear}(x) = xw + b$$

If $x$ is a mini-batch of shape $(bs, n_{inp})$ then $w$ has shape $(n_{inp}, 1)$ and $b$ has shape $(1,)$, with addition being done via broadcasting. When computing the gradient, we'll image that we just have a batch size of one 

$$x = [x_1, \dots, x_{n_{inp}}]$$ 

There are two gradients to compute this time around, one with respect to the weights and the other with respect to the bias. To make life easier, let's write things out in terms of coordinates:

$$z_i = \sum_{k=1}^{n_{inp}} x_k w_{ki} + b$$

Then we get

$$\frac{\partial \text{BCE}}{\partial w_{ki}} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}
{\partial w_{ki}} = x_k \frac{\partial \text{BCE}}{\partial z_i}$$

and

$$\frac{\partial \text{BCE}}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i}.$$

Remember, we'll already have the gradient of the loss with respect to the output $z_i$ of the linear layer stored in a variable $\text{grad}$ when it's time to compute the gradients with respect to the weights and biases. The nice thing is that $\text{grad}$ is exactly the gradient with respect to the $b$, so we only need to figure out how to write the gradient with respect to $w$ as a matrix product.

Whenever I have a hard time doing something like this, I just focus on getting the shapes right:

\begin{align}
  &\bullet x \text{ has shape } (bs, n_{inp}) \\
  &\bullet \text{grad has shape } (bs, 1) \\
  &\bullet \text{grad}_w \text{ has shape } (n_{inp}, 1)
\end{align}

The only way we can multiply $x$ and $\text{grad}$ and get something of shape $(n_{inp}, 1)$ is to resize $x$ to have shape $(bs, n_{inp}, 1)$ and $\text{grad}$ to have shape $(bs, 1, 1)$ so that ordinary matrix multiplication over the last two dimensions gives the shape $(n_{inp}, 1)$. The last thing to remember is that we average the gradients over the batch dimension to produce the final gradient updates.

Note that were there another linear layer we would also need to compute the gradient of the loss with respect to the inputs $x$ so that we could keep backpropagating the gradient, however, this isn't anymore complicated than what we've done so far, so let's do it anyways!

$$ \frac{\partial \text{BCE}}{x_k} = \sum_{k=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_k} \times \frac{\partial z_k}{\partial x_k} = \sum_{k=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_k} w_{ki} = \text{grad} \times w^t$$

As always, let's do a sanity check on the dimensions involved to make sure nothing has gone horribly wrong. Since $\text{grad}$ has shape $(bs, 1)$ and $w$ has shape $(n_{inp}, 1)$, $\text{grad} \times w^t$ has shape $(bs, n_{inp})$, which is exactly the shape of $x$--just as it should be.

In [5]:
class Linear:
    """Container for the forward and backward pass of a linear layer."""
    
    def __init__(self, n_inp, n_out):
        k = 1 / np.sqrt(n_inp)
        self.weights = np.random.uniform(-k, k, (n_inp, n_out))
        self.bias = np.zeros(n_out)
        
    def forward(self, x):
        """Pass a mini-batch through a linear layer."""
        self.x = x
        return x @ self.weights + self.bias
    
    def backward(self, grad):
        """Compute the gradients of the weights and biases given previous gradient."""
        self.grad_w = (self.x[:,:,None] @ grad[:,None,:]).mean(axis=0)
        self.grad_b = grad.mean(axis=0)
        return grad @ self.grad_w.T

---------

# Putting It All Together

Wohoo! It's finally time to string together all the work we've done so far into a single network.

In [6]:
class Sequential:
    """Container for a feedforward neural net."""
    
    def __init__(self, layers, criterion, metric):
        """Initialise layers, loss criterion, and evaluation metric."""
        self.layers = layers
        self.criterion = criterion
        self.metric = metric
        
    def forward(self, x):
        """Pass a mini-batch through the network."""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self):
        """Backpropagate gradients to the start of the network."""
        grad = self.criterion.backward()
        for layer in self.layers[::-1]:
            grad = layer.backward(grad)

In [7]:
class SGD:
    """Container for updating a model's weights via SGD."""
    
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
                  
    def step(self):
        """Update the weights and biases of all linear layers."""
        for layer in self.model.layers:
            if isinstance(layer, Linear):
                layer.weights -= self.lr * layer.grad_w
                layer.bias -= self.lr * layer.grad_b

----------

# Our Evaluation Metric

For simplicity, we'll just consider accuracy for the time being

In [8]:
def accuracy(y, y_hat):
    """Compute accuracy given soft binary predictions."""
    y_pred = y_hat > 0.5
    print(f"{y[:5]= } | {y_hat[:5]= } | {y_pred= }")
    return (y_pred == y).mean()

---------

# Trainer

To make life easier, let's wrap all of the functionality we need to train a network into a single class.

In [9]:
class Trainer:
    """Container for training a feedforward neural net."""
    
    def __init__(self, model, optimizer, train_dl, val_dl):
        self.model = model
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.val_dl = val_dl
        
    def _train(self):
        """Train for a single epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            self.model.backward()
            self.optimizer.step()
            loss += batch_loss
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for multiple epochs."""
        for epoch in range(n_epochs):
            loss = self._train()
            self.optimizer.step()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            batch_metric = self.model.metric(y, y_hat)
            metric += len(y) * batch_metric
            loss += batch_loss
            n += len(y)
        return loss / n, metric / n

---------

# Pre-process Data

We'll use sklearn's [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) for our binary classification task.

In [10]:
# Load data
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

((569, 30), (569,))

In [11]:
# Train-test-split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train.shape, X_val.shape

((455, 30), (114, 30))

In [12]:
# Normalize
mu, sigma = X_train.mean(), X_train.std()
X_train = (X_train - mu) / sigma
X_val = (X_val - mu) / sigma

---------

# Datasets & DataLoaders

In [13]:
class Dataset:
    """Container for returning inputs and targets."""
    
    def __init__(self, X, y):
        self.X = X
        self.y = y.reshape(-1, 1)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __setitem__(self, idx, val):
        self.X[idx], self.y[idx] = val
                
    def __len__(self):
        return len(self.y)

In [14]:
class DataLoader:
    """Container for returning a mini-batch of inputs and targets."""
    
    def __init__(self, ds, batch_size, shuffle=False):
        self.ds = ds
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def shuffle_data(self):
        """Shuffle inputs and targets."""
        idxs = np.random.permutation(len(self.ds))
        self.ds = Dataset(*self.ds[idxs])
        
    def __iter__(self):
        """Yield a mini-batch of inputs and targets."""
        if self.shuffle: self.shuffle_data()
        n_batches = len(self.ds) // self.batch_size
        for i in range(n_batches):
            yield self.ds[i * self.batch_size: (i + 1) * self.batch_size]

In [15]:
# Load training and validation data
train_ds = Dataset(X_train, y_train)
val_ds = Dataset(X_val, y_val)

batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(X_val), shuffle=False)

--------

# Train

Now we're ready to put our model to the test.

In [16]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 1), Sigmoid()]
model = Sequential(layers, criterion, metric)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.1)
trainer = Trainer(model, optimizer, train_dl, val_dl)

In [17]:
trainer.train(20)

self.y[:5]= array([[0],
       [1],
       [0],
       [1],
       [1]])
self.y_hat[:5]= array([[0.34681156],
       [0.3973389 ],
       [0.39874623],
       [0.39172407],
       [0.42278559]])
self.y[:5]= array([[0],
       [1],
       [1],
       [0],
       [1]])
self.y_hat[:5]= array([[0.38535726],
       [0.43795197],
       [0.40459576],
       [0.38069518],
       [0.4516522 ]])
self.y[:5]= array([[1],
       [1],
       [0],
       [1],
       [0]])
self.y_hat[:5]= array([[0.49762823],
       [0.47877652],
       [0.45489394],
       [0.49853388],
       [0.42176934]])
self.y[:5]= array([[1],
       [0],
       [1],
       [1],
       [0]])
self.y_hat[:5]= array([[0.46175839],
       [0.3300274 ],
       [0.46907292],
       [0.48907898],
       [0.4012842 ]])
self.y[:5]= array([[1],
       [1],
       [1],
       [0],
       [0]])
self.y_hat[:5]= array([[0.47493233],
       [0.43243539],
       [0.42013153],
       [0.40121868],
       [0.43326411]])
self.y[:5]= array([[1],
 

-------------

# Comparison with sklearn

Let's compare how our logistic network stacks up against sklearn's own logistic regression model.

In [18]:
# We're close!
sklearn_model = LogisticRegression(random_state=seed)
sklearn_model.fit(X_train, y_train)
sklearn_model.score(X_val, y_val)

0.9122807017543859

----------------

# Adding a Second Layer

Let's add another linear layer to our network, followed, of course, by an activation.

In [19]:
class ReLU:
    """Container for the forward and backward pass of ReLU."""
    
    def forward(self, x):
        """Pass a mini-batch through ReLU."""
        self.x = x
        return np.where(x > 0, x, 0)
    
    def backward(self, grad):
        """Return the gradient where x is positive, otherwise zero."""
        return np.where(self.x > 0, grad, 0)

In [20]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 20), ReLU(), Linear(20, 1), Sigmoid()]
model = Sequential(layers, criterion, accuracy)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.01)
trainer = Trainer(model, optimizer, train_dl, val_dl)

In [21]:
trainer.train(20)

self.y[:5]= array([[1],
       [1],
       [0],
       [1],
       [1]])
self.y_hat[:5]= array([[0.42738685],
       [0.44433219],
       [0.38498309],
       [0.48048508],
       [0.44960154]])
self.y[:5]= array([[0],
       [1],
       [1],
       [0],
       [0]])
self.y_hat[:5]= array([[0.39646119],
       [0.44330091],
       [0.46082886],
       [0.32570623],
       [0.3996871 ]])
self.y[:5]= array([[0],
       [1],
       [1],
       [1],
       [0]])
self.y_hat[:5]= array([[0.34185414],
       [0.44834042],
       [0.4570221 ],
       [0.44295038],
       [0.39871469]])
self.y[:5]= array([[0],
       [1],
       [1],
       [0],
       [1]])
self.y_hat[:5]= array([[0.43049624],
       [0.43712379],
       [0.4447248 ],
       [0.42083249],
       [0.45501087]])
self.y[:5]= array([[1],
       [0],
       [1],
       [1],
       [1]])
self.y_hat[:5]= array([[0.4439863 ],
       [0.404639  ],
       [0.45496764],
       [0.44335711],
       [0.47959196]])
self.y[:5]= array([[1],
 

self.y[:5]= array([[1],
       [1],
       [1],
       [1],
       [1]])
self.y_hat[:5]= array([[0.46216161],
       [0.48951161],
       [0.50579721],
       [0.47720095],
       [0.48128145]])
self.y[:5]= array([[1],
       [1],
       [1],
       [0],
       [1]])
self.y_hat[:5]= array([[0.48796354],
       [0.47440495],
       [0.49073345],
       [0.42370351],
       [0.50583476]])
self.y[:5]= array([[1],
       [1],
       [1],
       [0],
       [1]])
self.y_hat[:5]= array([[0.47726876],
       [0.49036269],
       [0.5052233 ],
       [0.45310461],
       [0.49142557]])
self.y[:5]= array([[1],
       [1],
       [1],
       [1],
       [1]])
self.y_hat[:5]= array([[0.49273293],
       [0.48242351],
       [0.49846869],
       [0.49761219],
       [0.51021239]])
self.y[:5]= array([[1],
       [1],
       [1],
       [1],
       [1]])
self.y_hat[:5]= array([[0.48882575],
       [0.48051496],
       [0.47465143],
       [0.48298855],
       [0.52160789]])
y[:5]= array([[0],
      

------------

# PyTorch Comparison

In [22]:
class PyTrainer:
    """Container for training a PyTorch feedforward neural net."""
    
    def __init__(self, model, optimizer, criterion, metric, train_dl, val_dl):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.metric = metric
        self.train_dl = train_dl
        self.val_dl = val_dl
        
    def _train(self):
        """Train for a single epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.criterion(y_hat, y)
            self.optimizer.zero_grad()
            batch_loss.backward()
            self.optimizer.step()
            loss += len(y) * batch_loss.item()
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for multiple epochs."""
        for epoch in range(n_epochs):
            self.model.train()
            loss = self._train()
            self.optimizer.step()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        self.model.eval()
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.criterion(y_hat, y)
            batch_metric = self.metric(y, y_hat)
            metric += len(y) * batch_metric
            loss += len(y) * batch_loss.item()
            n += len(y)
        return loss / n, metric / n

In [23]:
import torch
import torch.nn as nn

In [24]:
# Convert np -> tensor
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)

y_train = torch.FloatTensor(y_train)
y_val = torch.FloatTensor(y_val)

In [25]:
# Load training and validation data
train_ds = Dataset(X_train, y_train)
val_ds = Dataset(X_val, y_val)

batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(X_val), shuffle=False)

In [26]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise model
metric = accuracy
criterion = nn.functional.binary_cross_entropy
model = nn.Sequential(nn.Linear(n_inp, 20), nn.ReLU(), nn.Linear(20, 1), nn.Sigmoid())

# Initialise optimizer and trainer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
pytrainer = PyTrainer(model, optimizer, criterion, metric, train_dl, val_dl)

In [27]:
pytrainer.train(1)

y[:5]= tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]]) | y_hat[:5]= tensor([[0.5253],
        [0.5247],
        [0.5045],
        [0.5310],
        [0.5345]], grad_fn=<SliceBackward>) | y_pred= tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
 

RuntimeError: Can only calculate the mean of floating types. Got Bool instead.